In [ ]:
%matplotlib inline

import glob
import numpy as np

import keras
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten,BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import math

import dataset_load
import img_global

np.random.seed(0)


# model building
def build_model(keep_prob):
    print("Constructing model")
    model = Sequential()
    model.add(BatchNormalization(input_shape=IMG_SHAPE))
#   model.add(Lambda(lambda x: (x / 128 - 1), input_shape=IMG_SHAPE))
    model.add(Conv2D(24, (5, 5), activation='elu',strides=(2,2),kernel_initializer='lecun_normal',name='conv1'))
    model.add(Conv2D(32, (5, 5), activation='elu',strides=(2,2),kernel_initializer='lecun_normal',name='conv2'))
    model.add(Conv2D(48, (5, 5), activation='elu',strides=(2,2),kernel_initializer='lecun_normal',name='conv3'))
    model.add(Conv2D(64, (3, 3), activation='elu',kernel_initializer='lecun_normal',name='conv4'))
    model.add(Conv2D(64, (3, 3), activation='elu',kernel_initializer='lecun_normal',name='conv5'))
    
    #model.add(Conv2D(8,(3,3),activation='relu', kernel_initializer='he_normal'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(keep_prob))
    
    model.add(Flatten())
    model.add(Dense(100, activation='elu',name='fc2'))
    model.add(Dense(50, activation='elu',name='fc3'))
#   model.add(Dense(500, activation='elu',name='fc1'))
#   model.add(Dense(250, activation='elu',name='fc2'))
#   model.add(Dense(50, activation='elu',name='fc3'))  
    model.add(Dense(3, activation='softmax'))#Output layer, sigmoid may cause gradient vanishment
    model.summary()

    return model


# Model traininig
def train_model(model, learning_rate, nb_epoch, samples_per_epoch,
                batch_size, X_train, y_train):
    
    # Save the model after every epoch.
    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='min')
    # Stop training when loss of validation set has stopped improving.
    # After 'patience number' of epoch
    early_stop = EarlyStopping(monitor='loss', min_delta=.0005, patience=10,
                               verbose=1, mode='min')
    tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=True,
                              write_images=True, embeddings_freq=0, embeddings_layer_names=None,
                              embeddings_metadata=None)
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])
    datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.11) # set validation split
    model.fit_generator(batch_generator(datagen,X_train, y_train, 1,batch_size),
#                       steps_per_epoch= samples_per_epoch/batch_size
                        steps_per_epoch=int(math.floor(len(X_train)*0.89 / batch_size)),
                        epochs=nb_epoch,
                        max_queue_size=1,
                        validation_data=batch_generator(datagen,X_train, y_train, 0,batch_size),
                        validation_steps=int(math.floor(len(X_train)*0.11 / batch_size)),
                        callbacks=[tensorboard, checkpoint, early_stop],
                        verbose=2)



def batch_generator(datagen,X, y, train_flag,batch_size):
    if train_flag==1:
        return datagen.flow(X, y, batch_size=batch_size,subset="training",shuffle='True') # set as training data
    
    if train_flag==0:
        return datagen.flow (
        X, y,
        batch_size=batch_size,
        subset="validation",shuffle='True') # set as validation data

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
#     classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def main():
    # Printing hyperparameter

    print('-' * 30)
    print('Hyperparameters:')
    print('-' * 30)

    keep_prob = 0.999
    learning_rate = 0.0001
    nb_epoch = 100
    samples_per_epoch = 10000
    batch_size = 128

    print('keep_prob = ', keep_prob)
    print('learning_rate = ', learning_rate)
    print('nb_epoch = ', nb_epoch)
    print('samples_per_epoch = ', samples_per_epoch)
    print('batch_size = ', batch_size)
    print('-' * 30)

    # Load data
    X_train,y_train = dataset_load.data_load("./aug_train_set")
    X_test,y_test= dataset_load.data_load("./test_set")
    print("X_train shape:"+str(X_train.shape))
    print("y_train shape:"+str(y_train.shape))
    print("X_test shape:"+str(X_test.shape))
    print("y_test shape:"+str(y_test.shape))
    
    print("data loaded and augmented")
    # Compile the model
    model = build_model(keep_prob)
    # Save the model
    train_model(model, learning_rate, nb_epoch, samples_per_epoch, batch_size, X_train, y_train)
    print("Model trained")
   
    class_names=['forward','turn left', 'turn right']
    model_loaded = glob.glob('./*.h5')  # 
    for single_mod in model_loaded:
        model = load_model(single_mod)
    predict=model.predict(X_test)
    y_pred=np.argmax(predict,axis=1)
    y_test_index=np.argmax(y_test,axis=1)
    
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plot_confusion_matrix(y_test_index, y_pred, classes=class_names,
                          title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plot_confusion_matrix(y_test_index, y_pred, classes=class_names, normalize=True,
                          title='Normalized confusion matrix')

    plt.show()

if __name__ == '__main__':
    sleep(3600)
    main()








Using TensorFlow backend.


------------------------------
Hyperparameters:
------------------------------
('keep_prob = ', 0.999)
('learning_rate = ', 0.0001)
('nb_epoch = ', 100)
('samples_per_epoch = ', 10000)
('batch_size = ', 128)
------------------------------
NPZ list ready
442 turns in total 
['img_data', 'img_labels']
('Current npz', 1)
The 1 turn is finished
['img_data', 'img_labels']
('Current npz', 2)
The 2 turn is finished
['img_data', 'img_labels']
('Current npz', 3)
The 3 turn is finished
['img_data', 'img_labels']
('Current npz', 4)
The 4 turn is finished
['img_data', 'img_labels']
('Current npz', 5)
The 5 turn is finished
['img_data', 'img_labels']
('Current npz', 6)
The 6 turn is finished
['img_data', 'img_labels']
('Current npz', 7)
The 7 turn is finished
['img_data', 'img_labels']
('Current npz', 8)
The 8 turn is finished
['img_data', 'img_labels']
('Current npz', 9)
The 9 turn is finished
['img_data', 'img_labels']
('Current npz', 10)
The 10 turn is finished
['img_data', 'img_labels']
('Curre

The 112 turn is finished
['img_data', 'img_labels']
('Current npz', 113)
The 113 turn is finished
['img_data', 'img_labels']
('Current npz', 114)
The 114 turn is finished
['img_data', 'img_labels']
('Current npz', 115)
The 115 turn is finished
['img_data', 'img_labels']
('Current npz', 116)
The 116 turn is finished
['img_data', 'img_labels']
('Current npz', 117)
The 117 turn is finished
['img_data', 'img_labels']
('Current npz', 118)
The 118 turn is finished
['img_data', 'img_labels']
('Current npz', 119)
The 119 turn is finished
['img_data', 'img_labels']
('Current npz', 120)
The 120 turn is finished
['img_data', 'img_labels']
('Current npz', 121)
The 121 turn is finished
['img_data', 'img_labels']
('Current npz', 122)
The 122 turn is finished
['img_data', 'img_labels']
('Current npz', 123)
The 123 turn is finished
['img_data', 'img_labels']
('Current npz', 124)
The 124 turn is finished
['img_data', 'img_labels']
('Current npz', 125)
The 125 turn is finished
['img_data', 'img_labels']

The 225 turn is finished
['img_data', 'img_labels']
('Current npz', 226)
The 226 turn is finished
['img_data', 'img_labels']
('Current npz', 227)
The 227 turn is finished
['img_data', 'img_labels']
('Current npz', 228)
The 228 turn is finished
['img_data', 'img_labels']
('Current npz', 229)
The 229 turn is finished
['img_data', 'img_labels']
('Current npz', 230)
The 230 turn is finished
['img_data', 'img_labels']
('Current npz', 231)
The 231 turn is finished
['img_data', 'img_labels']
('Current npz', 232)
The 232 turn is finished
['img_data', 'img_labels']
('Current npz', 233)
The 233 turn is finished
['img_data', 'img_labels']
('Current npz', 234)
The 234 turn is finished
['img_data', 'img_labels']
('Current npz', 235)
The 235 turn is finished
['img_data', 'img_labels']
('Current npz', 236)
The 236 turn is finished
['img_data', 'img_labels']
('Current npz', 237)
The 237 turn is finished
['img_data', 'img_labels']
('Current npz', 238)
The 238 turn is finished
['img_data', 'img_labels']

The 338 turn is finished
['img_data', 'img_labels']
('Current npz', 339)
The 339 turn is finished
['img_data', 'img_labels']
('Current npz', 340)
The 340 turn is finished
['img_data', 'img_labels']
('Current npz', 341)
The 341 turn is finished
['img_data', 'img_labels']
('Current npz', 342)
The 342 turn is finished
['img_data', 'img_labels']
('Current npz', 343)
The 343 turn is finished
['img_data', 'img_labels']
('Current npz', 344)
The 344 turn is finished
['img_data', 'img_labels']
('Current npz', 345)
The 345 turn is finished
['img_data', 'img_labels']
('Current npz', 346)
The 346 turn is finished
['img_data', 'img_labels']
('Current npz', 347)
The 347 turn is finished
['img_data', 'img_labels']
('Current npz', 348)
The 348 turn is finished
['img_data', 'img_labels']
('Current npz', 349)
The 349 turn is finished
['img_data', 'img_labels']
('Current npz', 350)
The 350 turn is finished
['img_data', 'img_labels']
('Current npz', 351)
The 351 turn is finished
['img_data', 'img_labels']

The 7 turn is finished
['img_data', 'img_labels']
('Current npz', 8)
The 8 turn is finished
['img_data', 'img_labels']
('Current npz', 9)
The 9 turn is finished
['img_data', 'img_labels']
('Current npz', 10)
The 10 turn is finished
['img_data', 'img_labels']
('Current npz', 11)
The 11 turn is finished
['img_data', 'img_labels']
('Current npz', 12)
The 12 turn is finished
['img_data', 'img_labels']
('Current npz', 13)
The 13 turn is finished
['img_data', 'img_labels']
('Current npz', 14)
The 14 turn is finished
['img_data', 'img_labels']
('Current npz', 15)
The 15 turn is finished
['img_data', 'img_labels']
('Current npz', 16)
The 16 turn is finished
['img_data', 'img_labels']
('Current npz', 17)
The 17 turn is finished
['img_data', 'img_labels']
('Current npz', 18)
The 18 turn is finished
['img_data', 'img_labels']
('Current npz', 19)
The 19 turn is finished
['img_data', 'img_labels']
('Current npz', 20)
The 20 turn is finished
['img_data', 'img_labels']
('Current npz', 21)
The 21 turn


Epoch 00026: val_loss did not improve from 0.57490
Epoch 27/100
 - 210s - loss: 0.5963 - acc: 0.7499 - val_loss: 0.8150 - val_acc: 0.6832

Epoch 00027: val_loss did not improve from 0.57490
Epoch 28/100
 - 212s - loss: 0.5915 - acc: 0.7579 - val_loss: 0.8378 - val_acc: 0.6827

Epoch 00028: val_loss did not improve from 0.57490
Epoch 29/100
 - 212s - loss: 0.5863 - acc: 0.7609 - val_loss: 0.8713 - val_acc: 0.6853

Epoch 00029: val_loss did not improve from 0.57490
Epoch 30/100
 - 210s - loss: 0.5721 - acc: 0.7679 - val_loss: 0.9286 - val_acc: 0.6912

Epoch 00030: val_loss did not improve from 0.57490
Epoch 31/100
 - 210s - loss: 0.5714 - acc: 0.7689 - val_loss: 0.9511 - val_acc: 0.6827

Epoch 00031: val_loss did not improve from 0.57490
Epoch 32/100
 - 211s - loss: 0.5606 - acc: 0.7771 - val_loss: 0.9692 - val_acc: 0.6896

Epoch 00032: val_loss did not improve from 0.57490
Epoch 33/100
 - 210s - loss: 0.5623 - acc: 0.7777 - val_loss: 1.0157 - val_acc: 0.6825

Epoch 00033: val_loss did 

KeyboardInterrupt: 